In [50]:
import re, itertools
import numpy as np



### Data-related

db = open("../pgn_databases/test.pgn", "r")

print(db.read(10000).split("\n\n")[1::2])

f_dict = {"a": 0, "b": 1, "c": 2, "d": 3, "e": 4, "f": 5, "g": 6, "h": 7}

### Board-related

def init_board():
    board = np.zeros((8, 8, 6), dtype=np.int8)
    
    # Pawns
    board[1, :, 0] = -1
    board[6, :, 0] = 1
    
    # Non-royalty
    for j in range(3):
        board[0, [j, 7-j], j+1] = -1
        board[7, [j, 7-j], j+1] = 1
    
    # Queens
    board[0, 3, 4] = -1
    board[7, 3, 4] = 1
    
    # Kings
    board[0, 4, 5] = -1
    board[7, 4, 5] = 1
    
    return board

    
def unroll(board):
    return board.ravel()

    
    
### Move-related
def parse_move(move):
    
    # Remove check notation (+)
    pattern = re.compile("\+") 
    move = pattern.sub("", move)    
    
    piece_dict = {"R": 1, "N": 2, "B": 3, "Q": 4, "K": 5}
    ambig = None
    
    # Identify piece type
    if move[0].isupper():
        d = piece_dict[move[0]]
        move = move[1:]
    else:
        d = 0
        if "x" in move:
            ambig = move[0]
            move = move[2:]
    
    # Check for ambiguity in move
    if len(move) == 3:
        ambig = move[0]
        move = move[1:]
    elif len(move) > 3:
        return -1
    
    r, f = 8-int(move[1]), f_dict[move[0]]
    
    return [r, f, d, ambig]

    
    
def find_diags(to):
    """ Finds squares on both diagonals that target square lies on """
    r, f, d, ambig = to
    
    diags = set()
    
    # Down-left    
    r_, f_ = r, f
    while r_ >= 0 and f_ <= 7:
        diags.add((r_, f_))
        r_ -= 1
        f_ += 1
    # Up-right
    r_, f_ = r, f
    while f_ >= 0 and r_ <= 7:
        diags.add((r_, f_))
        r_ += 1
        f_ -= 1    
    # Down-right
    r_, f_ = r, f
    while max(r_, f_) <= 7:
        diags.add((r_, f_))
        r_ += 1
        f_ += 1
    # Up-left
    r_, f_ = r, f
    while min(r_, f_) >= 0:
        diags.add((r_, f_))
        r_ -= 1
        f_ -= 1  
        
    return list(diags)
    
    
    
def move_simple(to, player):
    """
    Wipes old position and adds new position of the single piece in a layer
    """
    r, f, d, ambig = to
    board[:, :, d] = np.multiply(board[:, :, d],
                                 np.array([board[:, :, d] != player]))
    board[r, f, d] = player
    
    
    
def move_KQ(to, player):
    move_simple(to, player=player)
    

    
def move_R(to, player):
    r, f, d, ambig = to
    
    # If there's only one piece of that type
    if np.array([board[:, :, d] == player]).sum() == 1:
        return move_simple(to, player)
    
    # Check to see if it's the only piece of its type on the rank
    elif np.array([board[:, f, d] == player]).sum() == 1:
        board[:, f, d] = np.multiply(board[:, f, d],
                                     np.array([board[:, f, d] != player]))
    
    # Check to see if it's the only piece of its type on the file    
    elif np.array([board[r, :, d] == player]).sum() == 1:
        board[r, :, d] = np.multiply(board[r, :, d],
                                     np.array([board[r, :, d] != player]))
                                     
    # Use the disambiguator            
    else:
        try:
            a = int(ambig)-1
            board[:, a, d] = np.multiply(board[:, a, d],
                                         np.array([board[:, a, d] != player]))
            
        except ValueError:
            a = rank_dict[ambig]
            board[a, :, d] = np.multiply(board[a, :, d],
                                         np.array([board[a, :, d] != player]))
        
    board[r, f, d] = player

    
    
def move_N(to, player):
    r, f, d, ambig = to
    
    # If there's only one piece of that type
    if np.array([board[:, :, d] == player]).sum() == 1:
        return move_simple(to, player)
    
    # Check to see if it's the only one in potential origin squares
    origins = [[2, 1], [2, -1], [1, 2], [1, -2],
               [-1, 2], [-1, -2], [-2, 1], [-2, -1]]
    
    count = 0
    
    for og in origins:
        r_, f_ = np.add([r, f], og)
        if 0 <= r_ <= 7 and 0 <= f_ <= 7:
            if board[r_, f_, d] == player:
                count += 1
                
    if count == 1:
        for og in origins:
            r_, f_ = np.add([r, f], og)
            if 0 <= r_ <= 7 and 0 <= f_ <= 7:
                if board[r_, f_, d] == player:
                    board[r_, f_, d] = 0
                    
    # Use the disambiguator            
    else:
        try:
            a = int(ambig)-1
            board[:, a, d] = np.multiply(board[:, a, d],
                                         np.array([board[:, a, d] != player]))
            
        except ValueError:
            a = rank_dict[ambig]
            board[a, :, d] = np.multiply(board[a, :, d],
                                         np.array([board[a, :, d] != player]))
    
    board[r, f, d] = player
        
    

    
def move_B(to, player):
    r, f, d, ambig = to
   
    # If there's only one piece of that type
    if np.array([board[:, :, d] == player]).sum() == 1:
        return move_simple(to, player)
    
    # Only one bishop on each square colour; so just wipe both diagonals        
    else:
        diags = find_diags(to)
        for square in diags:
            r_, f_ = square
            if board[r_, f_, 3] == player:
                board[r_, f_, 3] = 0
        
    board[r, f, d] = player
    
    
    
def move_P(to, player):
    r, f, d, ambig = to
    
    if ambig is not None:
        a = f_dict[ambig]
        board[r+player, a, d] = 0

    elif board[r+player, f, d] == player:
        board[r+player, f, d] = 0
        
    else:
        board[r+2*player, f, d] = 0
        
    board[r, f, d] = player

['1.c4 c5 2.Nf3 b6 3.Nc3 Bb7 4.e3 e6 5.d4 cxd4 6.exd4 Bb4 7.Bd3 Nf6 8.O-O h6\n9.d5 O-O 10.Nd4 Bxc3 11.bxc3 Na6 12.Qf3 Nc5 13.Bc2 Ba6 14.Re1 Bxc4 15.Bxh6\ngxh6 16.Qf4 Nh7 17.Re3 f5 18.Rg3+ Kh8 19.Nf3 Bxd5 20.Ne5 Qf6 21.Rg6 Qe7 \n22.Rxh6 Rg8 23.Ng6+ Rxg6 24.Rxg6 Rg8 25.Qe5+ Rg7 26.Re1 Nf8 27.Rg3 Ne4 28.\nBxe4 Bxe4 29.Ree3 d6 30.Qd4 e5 31.Qd1 Ne6 32.Rxg7 Nxg7 33.Qd2 Kg8 34.f3 \nBb7 35.f4 Be4 36.Rg3 Kf7 37.a4 exf4 38.Qxf4 Nh5 0-1', '1.c4 g6 2.Nc3 c5 3.g3 Bg7 4.Bg2 Nc6 5.Nf3 d6 6.O-O Bf5 7.a3 Qd7 8.Nd5 Rc8 \n9.Re1 Nf6 10.Nxf6+ Bxf6 11.d3 O-O 12.Rb1 a6 13.b4 cxb4 14.axb4 b5 15.cxb5 \naxb5 16.Bb2 Bxb2 17.Rxb2 Rc7 18.d4 d5 19.Bf1 Rfc8 20.Qd2 e6 21.h3 f6 22.\nRa1 Nd8 23.Rba2 Nf7 24.Ra6 Nd6 25.g4 Nc4 26.Qh6 Bc2 27.Ne1 Nd6 28.Qf4 Ne4 \n29.f3 g5 30.Qh2 Nd2 31.h4 Nb3 32.R1a3 Nxd4 33.hxg5 fxg5 34.e3 Nb3 35.Qe5 \nNd2 36.Rxe6 Bg6 37.Raa6 Rc1 38.Rxg6+ hxg6 39.Rxg6+ Kh7 40.Qxg5 1-0', '1.c4 Nf6 2.g3 c5 3.Bg2 Nc6 4.Nc3 e6 5.Nf3 d5 6.cxd5 Nxd5 7.O-O Be7 8.d4 \nO-O 9.Nxd5 exd5 10.dxc5 Bxc5 11.Bg5 Qd7 12.Qd3

In [40]:
board = init_board()
board[:, :, 5]

array([[ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0]], dtype=int8)

In [27]:
move_KQ(parse_move("Kd1"), 1)
board[:, :, 5]

array([[ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0]], dtype=int8)

In [31]:
move_N(parse_move("Nc3"), 1)
board[:, :, 2]

array([[ 0, -1,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0]], dtype=int8)

In [32]:
move_R(parse_move("Ra5"), -1)
board[:, :, 1]

array([[ 0,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  1]], dtype=int8)

In [33]:
move_B(parse_move("Bb4"), -1)
board[:, :, 3]

array([[ 0,  0, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  1,  0,  0]], dtype=int8)

In [41]:
move_P(parse_move("d3"), 1)
move_P(parse_move("f5"), -1)
move_P(parse_move("g4"), 1)
board[:, :, 0]

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1,  0, -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 1,  1,  1,  0,  1,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [52]:
move_P(parse_move("fxg4"), -1)
board[:, :, 0]

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1,  0, -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 1,  1,  1,  0,  1,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [51]:
parse_move("fxg4")

[4, 6, 0, 'f']

In [53]:
move_P(parse_move("f3"), 1)
move_P(parse_move("h3"), 1)
move_P(parse_move("hxg4"), 1)
board[:, :, 0]

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1,  0, -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0],
       [ 1,  1,  1,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)